### 30 Day Weather Forecast Data
In this code, data was fetched from climate forecast in 30 days (https://rapidapi.com/auth/sign-up?referral=/community/api/open-weather-map) for select cities, and stored in individual CSVs.


In [1]:
import pandas as pd
import requests
import json
import numpy as np
from datetime import date

In [2]:
cities=["New York", "San Francisco", "Jersey City", "Washington DC", "Buffalo", "Albany", "Yonkers", "Ithaca", "Elmira", "Monsey"]

for i in cities:
    url = "https://community-open-weather-map.p.rapidapi.com/climate/month"

    querystring = {"q":i}

    headers = {
        'x-rapidapi-host': "community-open-weather-map.p.rapidapi.com",
        'x-rapidapi-key': "b51cef6865msh5517863c11c5a5dp11f0b9jsnb64a91709b8f"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    content = response.content
    loaded = json.loads(content)
    df = pd.DataFrame(loaded["list"])
    dfav = pd.DataFrame({"temp" : [] })
    for row in range(df.shape[0]):
        dfbaby = pd.DataFrame({"temp" : [df.loc[row, "temp"]["average"]]})
        dfbaby.index = [row]
        dfav = pd.concat([dfav,dfbaby])
    
    df[["temp"]] = dfav

    df.columns = ["dt", "humidity", "pressure", "temp average", "wind_speed"]
    df.to_csv(i+ ".csv")

Furthermore, this data went through basic analysis to find: 
- The city that has the highest wind speed
- The city that has the lowest pressure
- The date and time of highest humidity in each city

In [3]:
cities=['New York', 'San Francisco', 'Jersey City', 'Washington DC', 'Buffalo', 'Albany', 'Yonkers', 'Ithaca', 'Elmira', 'Monsey']
windspeed = {}
pressure = {}
for i in cities:
    data = np.genfromtxt( i + ".csv", delimiter = ",", skip_header=1)
    t = max(data[:,5])
    e = min(data[:,3])
    windspeed[i] = [t]
    pressure[i] = [e]
windspeed
yo = 0
city = ''
minus = 0
city2 = ''
for i in cities:
    if windspeed[i][0] > yo:
        yo = windspeed[i][0]
        city = i
    if pressure[i][0] > minus:
        minus = pressure[i][0]
for i in cities:
    if pressure[i][0] < minus:
        minus = pressure[i][0]
        city2 = i
        
print("The city with the highest windspeed is " + city + " with a maximum windspeed of " + str(yo))
print("The city with the lowest pressure is " + city2 + " with a minimum pressure of " + str(minus))

The city with the highest windspeed is Buffalo with a maximum windspeed of 6.1
The city with the lowest pressure is Buffalo with a minimum pressure of 1009.67


In [4]:
humid = {}
for i in cities:
    data = np.genfromtxt( i + ".csv", delimiter = ",", skip_header=1)
    t = max(data[:,2])
    result = np.where(data[:,2] == t)
    time = data[result[0][0],1]
    timestamp = date.fromtimestamp(time)
    print("In the city of " + i + ", the day with the highest humidity (" + str(t) +") is ", end ="")
    print(timestamp)

In the city of New York, the day with the highest humidity (76.24) is 2022-02-06
In the city of San Francisco, the day with the highest humidity (80.76) is 2022-02-07
In the city of Jersey City, the day with the highest humidity (79.67) is 2022-02-06
In the city of Washington DC, the day with the highest humidity (77.54) is 2022-01-14
In the city of Buffalo, the day with the highest humidity (80.65) is 2022-02-09
In the city of Albany, the day with the highest humidity (81.27) is 2022-02-06
In the city of Yonkers, the day with the highest humidity (78.28) is 2022-02-06
In the city of Ithaca, the day with the highest humidity (82.37) is 2022-01-17
In the city of Elmira, the day with the highest humidity (82.37) is 2022-01-17
In the city of Monsey, the day with the highest humidity (80.24) is 2022-02-06


Finally, all of the data was brought together in one final csv.

In [5]:
newdf = pd.DataFrame({"humidity" : [], "pressure" : [], "temp average" : [], "wind_speed" : [], "city_name" : []})

for i in cities:
    data = pd.DataFrame(pd.read_csv(i + ".csv", index_col="dt"))
    del data["Unnamed: 0"]
    data["city_name"] = i 
    newdf = pd.concat([newdf, data])
newdf.to_csv("all.csv")